# load data

In [ ]:
%matplotlib inline

from netCDF4 import Dataset

import numpy as np
import matplotlib.pyplot as plt
import seaborn

root_data = '/gpfs/home/nonnenma/projects/seasonal_forecasting/data/pyrina'

print('\n NA-EU region \n')

# air pressure at mean sea level (North Atlantic & EU) ANOMALIES
nc_fn = root_data + '/msl_ERA20c_monthly_1900-2010.NA-EU.anomalies.nc'
tmp =  Dataset(nc_fn, 'r').variables['msl'].__array__()
msl_naeu_anomalies = tmp.data
assert not tmp.mask
print('msl      shape', msl_naeu_anomalies.shape)

# sea surface temperatures (North Atlantic & EU) ANOMALIES
nc_fn = root_data + '/sst_ERA20c_monthly_1900-2010.NA-EU.anomalies.nc'
tmp =  Dataset(nc_fn, 'r').variables['sst'].__array__()
sst_naeu_anomalies = tmp.data
sst_naeu_anomalies_mask = np.unique(tmp.mask, axis=0)
assert sst_naeu_anomalies_mask.shape[0] == 1
print('sst      shape', sst_naeu_anomalies.shape)

print('\n EU region \n')

# Volumetric soil water layer 1 (EU) ANOMALIES
nc_fn = root_data + '/swvl1_ERA20c_monthly_1900-2010.EU.anomalies.nc'
tmp =  Dataset(nc_fn, 'r').variables['swvl1'].__array__()
swvl1_eu_anomalies = tmp.data
assert not tmp.mask
print('swvl1    shape', swvl1_eu_anomalies.shape)

# Temperature at 2m (EU) ANOMALIES
nc_fn = root_data + '/t2m_ERA20c_monthly_1900-2010.EU.anomalies.nc'
tmp =  Dataset(nc_fn, 'r').variables['t2m'].__array__()
t2m_eu_anomalies = tmp.data
assert not tmp.mask
print('t2m      shape', t2m_eu_anomalies.shape)

# Temperature at 2m (EU) MV (mostly used for reference)
nc_fn = root_data + '/t2m_ERA20c_monthly_1900-2010.EU.mv.nc'
tmp =  Dataset(nc_fn, 'r').variables['t2m'].__array__()
t2m_eu = tmp.data
assert not tmp.mask
print('t2m (MV) shape', t2m_eu.shape)

print('\n TNA region \n')

# sea surface temperatures (TNA) ANOMALIES
nc_fn = root_data + '/sst_ERA20c_monthly_1900-2010.TNA.anomalies.nc'
tmp =  Dataset(nc_fn, 'r').variables['sst'].__array__()
sst_tna_anomalies = tmp.data
sst_tna_anomalies_mask = np.unique(tmp.mask, axis=0)
assert sst_tna_anomalies_mask.shape[0] == 1
print('sst      shape', sst_tna_anomalies.shape)


plt.figure(figsize=(16,8))

plt.subplot(1,3,1)
plt.imshow(t2m_eu[::12,:,:].mean(axis=0))
plt.title('avg t2m map for EU')

plt.subplot(1,3,2)
plt.imshow(sst_tna_anomalies_mask[0,:,:])
plt.title('TNA mask (SSTs))')

plt.subplot(1,3,3)
plt.imshow(sst_naeu_anomalies_mask[0,:,:])
plt.title('NA EU mask (SSTs)')
plt.show()


# time stamps
ts = Dataset(nc_fn, 'r').variables['time'].__array__().data


# training data time stamps
train_months, test_months = [3,4,5], [6,7,8]
y_total, y_train = len(ts)//12, 51
m_train = y_train * 12

idx_target_train = np.zeros((len(train_months), y_train), dtype=np.int)
idx_source_train = np.zeros((len(train_months), y_train), dtype=np.int)
idx_target_test = np.zeros((len(test_months), y_total - y_train), dtype=np.int)
idx_source_test = np.zeros((len(test_months), y_total - y_train), dtype=np.int)
for i,m in enumerate(train_months):
    idx_source_train[i,:] = np.arange(m, m_train, 12)           # 1900 - 1950
    idx_source_test[i,:]  = np.arange(m_train+m,  len(ts), 12)  # 1951 - 2010
for i,m in enumerate(test_months):
    idx_target_train[i,:] = np.arange(m, m_train, 12)           # 1900 - 1950
    idx_target_test[i,:]  = np.arange(m_train+m,  len(ts), 12)  # 1951 - 2010

# let's not miss a year
assert np.prod(idx_source_train.shape) + np.prod(idx_source_test.shape) == len(ts)/12 * len(train_months)
assert np.prod(idx_target_train.shape) + np.prod(idx_target_test.shape) == len(ts)/12 * len(test_months)


In [ ]:
n_latents = 5

# recreate CCA analysis
- Canonical correlation analysis to identify subspaces $A$, $B$ in source space $X$ and target space $Y$, respectively, such that $(AX)_i$ and $(BY)_i$ are maximally correlated.
- in a second step, establish a (linear) mapping from $BY \approx Q AX$ to predict $BY$ from $AX$.
- predict new $Y$ from $Y \approx B^\dagger Q AX$

In [ ]:
from sklearn.cross_decomposition import CCA

# predict T2ms in Summer from soil moisture levels in Spring
X = swvl1_eu_anomalies.reshape(len(ts), -1)[idx_source_train,:].mean(axis=0)
Y = t2m_eu_anomalies.reshape(len(ts), -1)[idx_target_train,:].mean(axis=0)

# get projections A'X, B'Y such that A'X and B'Y are maximally correlated
cca = CCA(n_components=n_latents, scale=False, max_iter=10000, tol=1e-8)
cca.fit(X, Y) 

# get time-course of projected data
X_c, Y_c = cca.transform(X, Y)

# learn linear regression Y_c = X_c * Q (Q will be optimal in least-squares sense)
Q = np.linalg.pinv(X_c).dot(Y_c)

if n_latents < 20:
    plt.figure(figsize=(12,7))
    for i in range(n_latents):
        plt.subplot(2, np.ceil(n_latents/2), i+1)
        plt.plot(Y_c[:,i], 'r-', label='Y_c true')
        plt.plot(X_c.dot(Q)[:,i], 'b--', label='Y_c pred')
        plt.legend() if i == 0 else None
        plt.ylabel('CC_i, i = ' + str(i+1))
    plt.subplot(2, np.ceil(n_latents/2), np.ceil(n_latents/2)*2)
    plt.imshow(Q)
    plt.title('Q such that Y_c = X_c * Q')
    plt.suptitle('temporal regression - learning to predict activity of canonical coefficients')
    plt.show()

# predict T2ms for test data (1951 - 2010)
X_f = cca.transform(swvl1_eu_anomalies.reshape(len(ts), -1)[idx_source_test,:].mean(axis=0))
Y_f = X_f.dot(Q)
out_pred = Y_f.dot(cca.y_loadings_.T)
out_true = t2m_eu_anomalies.reshape(len(ts), -1)[idx_target_test,:].mean(axis=0)
assert np.all(out_pred.shape == out_true.shape)

# evaluate prediction performance
anomaly_corrs = np.zeros(out_pred.shape[1])
for i in range(anomaly_corrs.size):
    anomaly_corrs[i] = np.corrcoef(out_pred[:,i], out_true[:,i])[0,1]
    
# visualize anomaly correlations
anomaly_corrs_map = anomaly_corrs.reshape(t2m_eu.shape[1], t2m_eu.shape[2])
cmax = np.max(np.abs(anomaly_corrs))
plt.imshow(anomaly_corrs_map, cmap='seismic', vmin=-cmax, vmax=cmax)
plt.colorbar()
plt.title(f'anomaly correlation coefficient, avg: {anomaly_corrs.mean():.3f}')
plt.show()

plt.figure(figsize=(16,8))
for i,y in enumerate([0, 10, 20, 30, 40, 50,]):
    plt.subplot(2,3, i+1)
    plt.imshow(np.hstack((out_pred[y,:].reshape(37,42), out_true[y,:].reshape(37,42))), cmap='gray')
    plt.ylabel(str(1900 + y_train + y))
plt.suptitle('example predictions (left: predicted T2ms, right: true T2ms)')
plt.show()

# simple low-rank linear prediction (pixel MSEs) 

- set up simple model $Y = W X$ with $W = U V$
- low-rank: if $Y \in \mathbb{R}^N, X \in \mathbb{R}^M$, then $W \in \mathbb{R}^{N \times M}$, but $U \in \mathbb{R}^{N \times k}, V \in \mathbb{R}^{k \times M}$ with $k << M,N$ !
- low-rank structure saves us parameters: $M N$ parameters in $W$, but only $N k + k M$ in $U$ and $V$, helps prevent overfitting on low samples size

In [ ]:
from sklearn.decomposition import PCA

# analytic expression for gradients
# (could use autodiff packages such as pytorch, tensorflow instead, but currently not on cluster 'Strand'...)
# Y = f(X) = W * X = U * V * X
# L_i =  || Y_i - f(X_i) ||^2 = (Y_i -f(X_i))' * (Y_i -f(X_i)) = Y_i'Y_i - 2 Y_i' f(X_i) + f(X_i)'f(X_i)
#     = - 2 Y_i' U V X_i + X_i' V' U' U V X_i + const.
# dL_i/dU = -2 Y_i X_i' V' + 2 * U V X_i X_i' V'
# dL_i/dV = -2 U' Y_i X_i' + 2 * U' U V X_i X_i'


# predict T2ms in Summer from soil moisture levels in Spring
X = swvl1_eu_anomalies.reshape(len(ts), -1)[idx_source_train,:].mean(axis=0)
Y = t2m_eu_anomalies.reshape(len(ts), -1)[idx_target_train,:].mean(axis=0)

# loss function and gradients (loss just for tracking convergence of gradient descent)

def L(UV):
    """
    Froebenius norm between targets Y and prediction W X = U V X
        L(X,Y,V,U) = || Y - U V X ||**2
    """
    U,V = UV[:n_latents, :], UV[n_latents:, :] # unpack parameters

    return np.sum((Y - X.dot(V.T).dot(U))**2)

def dL(X,Y,UV):
    """
    Gradients w.r.t U, V for  
        L(X,Y,V,U) = || Y - U V X ||**2
    """
    U,V = UV[:n_latents, :], UV[n_latents:, :] # unpack parameters
    VX, UY = V.dot(X), U.dot(Y) # prioritize shrinking sizes 
    VXX = VX.dot(X.T)           # over reusing computations
    dLdU = U.T.dot(VXX.dot(V.T)) - Y.dot(VX.T)
    dLdV = U.dot(U.T).dot(VXX) - UY.dot(X.T)
    
    return 2 * np.vstack((dLdU.T, dLdV))
    
def grad(params): 
    return dL(X.T,Y.T,params) # decorated gradient to avoid always passing X,Y

# initialize guesses for V,U from PCA of respective X,Y spaces
pca_X = PCA(n_components=n_latents).fit(X)
pca_Y = PCA(n_components=n_latents).fit(Y)
U_init, V_init = pca_Y.components_, pca_X.components_

# poor man's gradient descent ...
lr = 0.00001                                # learning rate
loss = np.zeros(100000)                     # container for loss values
U, V = U_init.copy(), V_init.copy()
UV = np.vstack((U,V))
for t in range(len(loss)):
    loss[t] = L(UV)
    UV -= lr * grad(UV)
U,V = UV[:n_latents, :], UV[n_latents:, :]

plt.plot(loss[len(loss)//2:])
plt.title('loss curve for gradient descent on U, V')
plt.show()
    
# predict T2ms for test data (1951 - 2010)
X_f = swvl1_eu_anomalies.reshape(len(ts), -1)[idx_source_test,:].mean(axis=0)
out_pred = X_f.dot(V.T).dot(U)
out_true = t2m_eu_anomalies.reshape(len(ts), -1)[idx_target_test,:].mean(axis=0)
assert np.all(out_pred.shape == out_true.shape)

# evaluate prediction performance
anomaly_corrs = np.zeros(out_pred.shape[1])
for i in range(anomaly_corrs.size):
    anomaly_corrs[i] = np.corrcoef(out_pred[:,i], out_true[:,i])[0,1]
    
# visualize anomaly correlations
anomaly_corrs_map = anomaly_corrs.reshape(t2m_eu.shape[1], t2m_eu.shape[2])
cmax = np.max(np.abs(anomaly_corrs))
plt.imshow(anomaly_corrs_map, cmap='seismic', vmin=-cmax, vmax=cmax)
plt.colorbar()
plt.title(f'anomaly correlation coefficient, avg: {anomaly_corrs.mean():.3f}')
plt.show()

plt.figure(figsize=(16,8))
for i,y in enumerate([0, 10, 20, 30, 40, 50,]):
    plt.subplot(3,2, i+1)
    out_init = X_f[y,:].dot(V_init.T).dot(U_init).reshape(37,42)
    plt.imshow(np.hstack((out_pred[y,:].reshape(37,42), out_true[y,:].reshape(37,42), out_init)), cmap='gray')
    plt.ylabel(str(1900 + y_train + y))
plt.suptitle('example predictions (left: predicted T2ms, center: true T2ms, right: PCA initializations)')
plt.show()

# debug